## MeSH Tree

In [37]:
import json
import sys
import time
import pandas as pd

In [38]:
meshtree = "data/mtrees2018.bin"  # to change to mtrees2017.bin
m2pfile ="data/mesh2pmid_stat.json"

In [39]:
with open(m2pfile, 'r') as ff:
    m2p = json.load(ff)

In [40]:
csvTree = []
id2name = {}


with open(meshtree, "r") as ftree:
    for line in ftree:
        term_tree = line.strip().split(";")
        cur_term = term_tree[0]
        cur_tree = term_tree[1]
        
        
        csvTree.append({'id':cur_tree ,'name':cur_term})
        id2name.update({cur_tree : cur_term})
        
        

In [41]:
ROOTS = {"Cardiovascular Diseases": "C14"}

for key,value in ROOTS.items(): 
    fname ="CVD"
    root = value
    idata = []
    r = root.split('.')
    ln = len(r)

    # first node   
    nm = id2name[r[0]]
    tt = r[0]
    #idata.append({'id': nm, 'value': tt})
       
    # remaining nodes   
    for t in r[1:]:
        tt = tt + '.' + t
        nm = nm + '.' +  id2name[tt]
        #idata.append({'id': nm, 'value': tt})
        
        
                
    # find root name <---------------------------------------
    root_name = id2name[r[0]] 
    tt = r[0]
    for t in r[1:]:
        tt = tt + '.' + t
        root_name = root_name + '.' +  id2name[tt]
        
        
            
    # created intended tree <-------------------------------------
    for key,values in id2name.items():
        
        items = key.split('.')
        ir = items[0]
        
        for t in items[1:ln]:
            ir = ir + '.' + t
            
        if len(items) > ln and ir == root:
                    
                    ID = root
                    l = len(items)
                    
                    for i in range(ln,l):
                        ID = ID + '.' + items[i]
                        #print(items,ID)
                        new_name  = id2name[ID].replace(", ","|")
                        new_name  = new_name.replace("-","~")
                        new_name = new_name.replace(" ","_")
                        
                        if i == ln:
                            name =  new_name
                            
                        else:
                            #print(id2name[ID].replace(", ", "_"))
                            
                            if len(id2name[ID].split())> 2:
                                name = name + '.' + new_name 
                            else:
                                name = name + '.' + new_name
                        
                        
                    idata.append({'id': str(name),'value': ID})
                
                
                    
                
                
    # save file per root <-------------------------------------------
    SM = pd.DataFrame(idata)
    SM = SM.sort_values( ['value'],axis =0,ascending =True)
    SM = SM.set_index('id')
    SM.to_csv('data/' + fname + '.csv') 

In [42]:
sid = list(SM.index)
sval = list(SM['value'])
allid = []
for t in sval:
    allid.append(t)
uid = allid
len(uid)

638

### Unique

In [46]:
for item in allid:
    tt = item.split(".")
    term =tt[0]
    for it in tt[1:-1]:
            term = term +"." + it
            if term in uid:
                uid.remove(term)
                #print(term,"removed left ",len(uid))

print(len(uid), "run again")

457 run again


In [47]:
DATA = []
for u_id in uid:
    for v,i in zip(sval,sid):
        if u_id == v:
            
            last = i.split(".")[-1]
            last = last.replace("_"," ")
            last = last.replace("~","-")
            last = last.replace("|",", ")
            
            #print(last)
            count = m2p[last]
            
            DATA.append({"id": i.replace(".","-"),"nid":v,"value":count})

In [48]:
DF = pd.DataFrame(DATA)
DF = DF.sort_values(['nid'])
DF = DF.drop(['nid'], axis =1)

In [49]:
DF = DF.set_index('id')
DF.to_csv('data/Flare.csv') 

### Color

In [50]:
ALL = []
for item in list(DF.index):
    LL = item.split("-")
    for item in LL:
        if item not in ALL:
            ALL.append(item)

In [51]:
len(ALL)

424

In [52]:
import random as random
Colors = {}
for item in ALL:
    color = "%06x" % random.randint(0, 0xFFFFFF)
    Colors.update({item: "#"+color})

In [53]:
Colors

{'Cardiovascular_Abnormalities': '#30f7fa',
 'Heart_Defects|Congenital': '#9c09a3',
 '22q11_Deletion_Syndrome': '#4a6835',
 'DiGeorge_Syndrome': '#15c9f3',
 'Alagille_Syndrome': '#47b90e',
 'Aortic_Coarctation': '#5bdeef',
 'Arrhythmogenic_Right_Ventricular_Dysplasia': '#a83630',
 'Barth_Syndrome': '#04d4d5',
 'Cor_Triatriatum': '#9859b5',
 'Coronary_Vessel_Anomalies': '#4fd95f',
 'Bland_White_Garland_Syndrome': '#62773d',
 'Myocardial_Bridging': '#dfa7a9',
 'Crisscross_Heart': '#af1cc9',
 'Dextrocardia': '#57fb26',
 'Kartagener_Syndrome': '#a0d74d',
 'Ductus_Arteriosus|Patent': '#5368be',
 'Ebstein_Anomaly': '#6c1cd9',
 'Ectopia_Cordis': '#589565',
 'Eisenmenger_Complex': '#7b2d1b',
 'Heart_Septal_Defects': '#28653c',
 'Aortopulmonary_Septal_Defect': '#d85b59',
 'Truncus_Arteriosus|Persistent': '#dbbfed',
 'Endocardial_Cushion_Defects': '#b30ec6',
 'Heart_Septal_Defects|Atrial': '#01fd98',
 'Foramen_Ovale|Patent': '#27c0af',
 'Lutembacher_Syndrome': '#20e6e3',
 'Heart_Septal_Defects|V

In [54]:
with open("data/color.json",'w') as fw:
    json.dump(Colors,fw)